In [1]:
!pip install transformers datasets torch
!pip install --upgrade datasets
!pip install evaluate
!pip install seqeval


[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import DataCollatorForTokenClassification, CamembertTokenizerFast, CamembertForTokenClassification, Trainer, TrainingArguments
import evaluate

# Chargement des données d'entraînement et de test dans un DataFrame pandas
train_data = pd.read_csv('dataset_phrases_labelled.csv')
test_data = pd.read_csv('dataset_phrases_labelled_test.csv')

# Convertir les chaînes de listes en listes réelles pour les deux ensembles
train_data['tokens'] = train_data['tokens'].apply(eval)
train_data['labels'] = train_data['labels'].apply(eval)
test_data['tokens'] = test_data['tokens'].apply(eval)
test_data['labels'] = test_data['labels'].apply(eval)

# Créer un dictionnaire pour les étiquettes
label_list = ['O', 'B-depart', 'I-depart', 'B-arrive', 'I-arrive', 'B-etape', 'I-etape']
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}

# Verification et convertion les labels en identifiants numériques
def check_labels(labels):
    unknown_labels = [label for label in labels if label not in label_to_id]
    if unknown_labels:
        print(f"Étiquettes inconnues trouvées: {unknown_labels}")
    return not unknown_labels

valid_train_labels = train_data['labels'].apply(check_labels)
valid_test_labels = test_data['labels'].apply(check_labels)

if valid_train_labels.all() and valid_test_labels.all():
    train_data['labels'] = train_data['labels'].apply(lambda x: [label_to_id[label] for label in x])
    test_data['labels'] = test_data['labels'].apply(lambda x: [label_to_id[label] for label in x])
else:
    print("Certaines étiquettes dans les données ne sont pas définies dans label_list.")

# Tokenizer Camembert
tokenizer = CamembertTokenizerFast.from_pretrained("camembert-base")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True, padding="max_length", max_length=128)
    labels = []
    for i, label in enumerate(examples['labels']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Récupère l'alignement des mots
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # ignore ce token pour la loss
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)  # Ignore les sous-tokens
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Convertir les DataFrames en datasets Hugging Face
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(test_data)

# Appliquer la tokenisation et l'alignement des labels
tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_and_align_labels, batched=True)

# Charger le modèle Camembert pour la classification de tokens
model = CamembertForTokenClassification.from_pretrained("camembert-base", num_labels=len(label_list))

# Préparer le collator de données pour gérer le padding des séquences
data_collator = DataCollatorForTokenClassification(tokenizer)

metric = evaluate.load("seqeval")
# Calcul des métriques
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)
    true_predictions = [
        [id_to_label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id_to_label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialiser Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1875 [00:00<?, ?it/s]

...

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.06810569018125534, 'eval_precision': 0.9858686616791354, 'eval_recall': 0.9868941127522364, 'eval_f1': 0.9863811206986174, 'eval_accuracy': 0.9974204974204974, 'eval_runtime': 20.0114, 'eval_samples_per_second': 99.943, 'eval_steps_per_second': 6.246, 'epoch': 2.0}
{'loss': 0.0743, 'grad_norm': 0.22617383301258087, 'learning_rate': 4.000000000000001e-06, 'epoch': 2.4}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.05899197235703468, 'eval_precision': 0.983364524849241, 'eval_recall': 0.9837736634075307, 'eval_f1': 0.9835690515806987, 'eval_accuracy': 0.9967659967659968, 'eval_runtime': 20.3904, 'eval_samples_per_second': 98.085, 'eval_steps_per_second': 6.13, 'epoch': 3.0}
{'train_runtime': 1065.0581, 'train_samples_per_second': 28.167, 'train_steps_per_second': 1.76, 'train_loss': 0.17525142008463543, 'epoch': 3.0}


TrainOutput(global_step=1875, training_loss=0.17525142008463543, metrics={'train_runtime': 1065.0581, 'train_samples_per_second': 28.167, 'train_steps_per_second': 1.76, 'total_flos': 1959814264320000.0, 'train_loss': 0.17525142008463543, 'epoch': 3.0})

In [17]:
import torch

def predict_sentence(sentence, model, tokenizer, label_list):
    if isinstance(sentence, str):
        words = sentence.split()
    else:
        words = sentence

    # Tokeniser les mots avec alignement
    tokenized_inputs = tokenizer(words, truncation=True, padding="max_length", max_length=128, is_split_into_words=True, return_tensors="pt")
    word_ids = tokenized_inputs.word_ids()  # Alignement des mots

    # Déplacer les tensors d'entrée vers le même périphérique que le modèle
    tokenized_inputs = {key: tensor.to(model.device) for key, tensor in tokenized_inputs.items()}

    # Passer les données dans le modèle
    with torch.no_grad():
        output = model(**tokenized_inputs)

    # Prédictions et extraction des labels
    predictions = output.logits.argmax(dim=-1).squeeze().cpu().tolist()
    tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"].squeeze().cpu().tolist())

    # Fusionner les sous-tokens
    result = []
    current_word = ""
    current_label = None
    for token, label_id, word_id in zip(tokens, predictions, word_ids):
        if token == "<pad>" or word_id is None:
            continue

        label = label_list[label_id]
        if token.startswith("▁"):
            if current_word:
                result.append((current_word, current_label))
            current_word = token[1:]
            current_label = label
        else:
            current_word += token

    if current_word:
        result.append((current_word, current_label))

    return result


In [20]:
import ipywidgets as widgets
from IPython.display import display

text_input = widgets.Text(
    value='',
    placeholder='Entrez une phrase pour la prédiction...',
    description='Phrase:',
    disabled=False
)

output = widgets.Output()

def on_text_change(change):
    with output:
        output.clear_output()
        sentence = change['new']
        result = predict_sentence(sentence, model, tokenizer, label_list)
        print("Tokens et labels prédits :")
        for token, label in result:
            print(f"{token}: {label}")

text_input.observe(on_text_change, names='value')

display(text_input, output)


Text(value='', description='Phrase:', placeholder='Entrez une phrase pour la prédiction...')

Output()

In [19]:
trainer.evaluate()

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.05899197235703468,
 'eval_precision': 0.983364524849241,
 'eval_recall': 0.9837736634075307,
 'eval_f1': 0.9835690515806987,
 'eval_accuracy': 0.9967659967659968,
 'eval_runtime': 21.4822,
 'eval_samples_per_second': 93.1,
 'eval_steps_per_second': 5.819,
 'epoch': 3.0}